In [27]:
import bempp.api 
import numpy as np
import mtf

from mtf.utils import bhmie
from mtf.config import config
from mtf.functions import define_bempp_functions

from matplotlib import pyplot as plt
from decimal import Decimal

bempp.api.enable_console_logging()
M = 2

mtf.config.set_case("B")

tangential_trace, neumann_trace = define_bempp_functions(config)

#k_ext, k_int = config["k_ext"], config["k_int"]
k0, k1 = config["k_ext"], config["k_int"]
lambda_par, freq = config["lambda"], config["frequency"]

polarization = config["polarization"]
direction = config["direction"]

eps_rel = config["eps_rel"]
mu_rel = config["mu_rel"]
mu0 = config["mu_ext"]
mu1 = mu_rel * mu0

eta_rel = np.sqrt(mu_rel / eps_rel)

print("The exterior wavenumber is: {0}".format(k0))
print("The interior wavenumber is: {0}".format(k1))

print("----")
print("The exterior wavelenght is: {0}".format(lambda_par))
print("The exterior frequency is: {:.2E}".format(Decimal(freq)))

The exterior wavenumber is: 5.0
The interior wavenumber is: 6.892024376045111
----
The exterior wavelenght is: 1.2566370614359172
The exterior frequency is: 2.39E+8


In [53]:
segments = [[10], [10]]
swapped_normals = [[10], []]

k_int, k_ext = config["k_int"], config["k_ext"]

n = k_int / k_ext
refIndex = n
numAngles = 901
s1, s2, qext, qsca, qback, gsca = bhmie(k_ext, k_int / k_ext, numAngles)
angles = config['angles']

#transmission_operators = assemble_operators(grid, config)
#far_field, solution = evaluate_far_field(transmission_operators, config)

k_list = [k0]
eta_rel_list = [1]
mu_list = [mu0]

for index in range(M-1):
  k_list.append(k1)
  mu_list.append(mu1)
  eta_rel_list.append(eta_rel)

from bempp.api.assembly.boundary_operator import BoundaryOperator
from bempp.api.operators.boundary.maxwell import osrc_mte

class osrcMtE(BoundaryOperator):
    def __init__(self, wf, domain, range_, dual_to_range, parameters):
        self.wf = wf
        self._domain = domain
        self._range = range_
        self._dual_to_range = dual_to_range
        self._parameters= parameters
        
    def weak_form(self):
        return self.wf
    

def osrc_MtE(grid, wave_number, parameters=None, type=1, npade=2):
    rwg = bempp.api.function_space(grid, 'RWG', 0)
    snc = bempp.api.function_space(grid, 'SNC', 0)
    p1d = bempp.api.function_space(grid, 'DP', 1)
    mte = bempp.api.operators.boundary.maxwell.osrc_mte([snc,p1d],  [snc, p1d],  [snc, p1d], wave_number, npade=npade, parameters=parameters, type=type)
    wf = mte._assemble()
    return osrcMtE(wf, rwg, rwg, snc, parameters)


def multitrace_osrc(grid, wave_number, parameters=None, type=1, npade=2):
    """Assemble the multitrace osrc operator for Maxwell."""

    blocked_operator = bempp.api.BlockedOperator(2, 2)
    osrc = osrc_MtE(grid, wave_number, parameters=None, type=type, npade=npade)

    blocked_operator[0, 1] = osrc
    blocked_operator[1, 0] = -1 * osrc

    return blocked_operator


In [63]:
mode = 'dense'

if mode == 'dense':
    assembler = None
elif mode == 'fmm':
    assembler = 'fmm'
    
precision = 1

h = 2 * np.pi/(precision*k0)
grid = bempp.api.shapes.sphere(h=h)


print(h, ': h')
print(precision, ': precision')
print(grid.number_of_edges * 2, ': N')

    
dA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]
rA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]
tA = [bempp.api.function_space(grid, "SNC", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

multitrace_ops = []

if assembler == "fmm": osrc_ops = []

# > Assemble all diagonal operators
for index in range(M):
  k = k_list[index]
  mu = mu_list[index]
  eta = eta_rel_list[index]
  efie = bempp.api.operators.boundary.maxwell.electric_field(dA[1], rA[1], tA[1], k, assembler=assembler)
  mfie = bempp.api.operators.boundary.maxwell.magnetic_field(dA[1], rA[1], tA[1], k, assembler=assembler)

  osrc = multitrace_osrc(multitrace_osrc(grid, k)
                         
  multitrace_ops.append(bempp.api.GeneralizedBlockedOperator([[mfie, eta * efie],[- 1/eta * efie, mfie]]), assembler=assembler)

lhs_op = multitrace_ops[0] + multitrace_ops[1]    
rhs = [bempp.api.GridFunction(rA[1], dual_space = tA[1], fun=tangential_trace),
      bempp.api.GridFunction(rA[1], dual_space = tA[1], fun=neumann_trace)]

##
if mode == 'dense':
    A = bempp.api.as_matrix(lhs_op.weak_form())
    b = bempp.api.assembly.blocked_operator.projections_from_grid_functions_list(rhs, lhs_op.dual_to_range_spaces)
    x = np.linalg.solve(A, b)
    sol = bempp.api.assembly.blocked_operator.grid_function_list_from_coefficients(x.ravel(), lhs_op.domain_spaces)
elif mode == 'primal':
    A = lhs_op.weak_form()
    sol = bempp.api.linalg.gmres(A)

far_field_points = config['far_field_points']
electric_far = bempp.api.operators.far_field.maxwell.electric_field(sol[1].space, far_field_points, k0)
magnetic_far = bempp.api.operators.far_field.maxwell.magnetic_field(sol[0].space, far_field_points, k0)    
far_field = - electric_far * sol[1] - magnetic_far * sol[0]

A22 = far_field[2,:]
uh = 10 * np.log10(4 * np.pi * np.abs(A22[:1801]))
u =  10 * np.log10(4 * np.pi * np.abs(s1 / (-1j * k_ext) ))
rel_error = np.linalg.norm(uh - u) / np.linalg.norm(u)

print(rel_error)

IndentationError: expected an indented block (4106612601.py, line 53)